In [1]:
from hana_ml import dataframe
from hana_ml.algorithms.pal.utility import DataSets
from gen_ai_hub.proxy.langchain import init_llm
llm = init_llm('gpt-4.1', temperature=0.0, max_tokens=512)

url="6291b51a-ea33-4ade-bb5c-0b8f1a032b9d.hana.demo-hc-3-hna0-hc-dev.dev-aws.hanacloud.ondemand.com"
port= 443
user="PALDEVUSER"
password="Abcd1234"
cc = dataframe.ConnectionContext(address=url, port=port, user=user, password=password)

In [6]:
import numpy as np
import pandas as pd
np.random.seed(42)
z = np.zeros(100)
pos = np.sort([0] + list(np.random.randint(1, 98, size=8)) + [99])
z[pos] = np.random.rand(10)

In [8]:
interm_data = pd.DataFrame(dict(DATE=pd.date_range(start='2025-01-01', periods=100),
                                ID=range(100),
                                DEMAND=z))

In [9]:
from hana_ml.dataframe import create_dataframe_from_pandas
interm_df = create_dataframe_from_pandas(cc, interm_data, '#MOCK_INTERMITTENT_DATA_TBL', force=True)

100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


In [10]:
from hana_ai.smart_dataframe import SmartDataFrame
from hana_ai.tools.df_tools.intermittent_forecast_tools import IntermittentForecast
from hana_ai.tools.df_tools.fetch_tools import FetchDataTool
sdf = SmartDataFrame(interm_df)
fetch_tool = FetchDataTool(connection_context=cc, return_direct=True)
interm_tool = IntermittentForecast(connection_context=cc, return_direct=False)
sdf.configure(tools=[fetch_tool, interm_tool], llm=llm)
sdf.ask("show me the first 5 rows of the dataframe")

,DATE,ID,DEMAND
0,2025-01-01,0,0.058084
1,2025-01-02,1,0.000000
2,2025-01-03,2,0.000000
3,2025-01-04,3,0.000000
4,2025-01-05,4,0.000000


In [11]:
sdf.ask("Please forecast the future values of DEMAND, where key is DATE and forecast horizon is 5.")

,DATE,DEMAND
0,2025-04-11,0.000000
1,2025-04-12,0.000000
2,2025-04-13,0.000000
3,2025-04-14,0.000000
4,2025-04-15,0.444934


In [ ]:
sdf.ask("Please forecast the future 4 values of DEMAND using consant method, where key is ID.")

,ID,OUTPUT_VALUE
0,100,0.092684
1,101,0.092684
2,102,0.092684
3,103,0.092684
